In [2]:
#No GUI
import requests

# Location, Temperature, Description, Humidity, Wind speed
# API key: b5c8d5d8a33d41eaacac223656d960f2
appid = 'b5c8d5d8a33d41eaacac223656d960f2'

# Classes:

# Create classes
class WeatherAPI:
    def __init__(self, appid):
        self.appid = appid

    def find_city(self, location_input):
        try:
            if ' ' in location_input:
                lat, lon = map(int, location_input.split())
                res = requests.get("http://api.openweathermap.org/data/2.5/find",
                                   params={
                                       'lat': lat,
                                       'lon': lon,
                                       'type': 'like',
                                       'units': 'metric',
                                       'APPID': self.appid
                                   })
            else:
                res = requests.get("http://api.openweathermap.org/data/2.5/find",
                                   params={
                                       'q': location_input,
                                       'type': 'like',
                                       'units': 'metric',
                                       'APPID': self.appid
                                   })

            data = res.json()
            return data['list'][0]['id'] if data['list'] else None
        except Exception as e:
            print("Exception (find):", e)
            return None

    def get_weather(self, city_id):
        try:
            res = requests.get("http://api.openweathermap.org/data/2.5/weather",
                               params={
                                   'id': city_id,
                                   'units': 'metric',
                                   'lang': 'en',
                                   'APPID': self.appid
                               })
            data = res.json()
            return {
                'temperature': data['main']['temp'],
                'description': data['weather'][0]['description'],
                'humidity': data['main']['humidity'],
                'wind_speed': data['wind']['speed']
            }
        except Exception as e:
            print("Exception (weather):", e)
            return None

class WeatherData:
    def __init__(self, temperature, description, humidity, wind_speed):
        self.temperature = temperature
        self.description = description
        self.humidity = humidity
        self.wind_speed = wind_speed

    def display_weather(self):
        print("Temperature: {}°C".format(self.temperature))
        print("Description: {}".format(self.description))
        print("Humidity: {}%".format(self.humidity))
        print("Wind Speed: {} km/h".format(self.wind_speed))

class Weather5(WeatherAPI):
    def __init__(self, appid):
        super().__init__(appid)

    def get_weather_5(self, city_id):
        try:
            res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                               params={
                                   'id': city_id,
                                   'units': 'metric',
                                   'lang': 'en',
                                   'APPID': self.appid
                               })
            data = res.json()
            print("5-Day Forecast:")
            unique_days = set()
            for day in data['list']:
                # Check if the time is 12:00:00
                if day['dt_txt'].endswith('12:00:00'):
                    date = day['dt_txt'].split(' ')[0]
                    if date not in unique_days:
                        unique_days.add(date)
                        print(f"Day: {day['dt_txt']}, Temperature: {day['main']['temp']}°C, Description: {day['weather'][0]['description']}")
        except Exception as e:
            print("Exception (forecast):", e)

class WeatherApp:
    def __init__(self, appid):
        self.weather_api = WeatherAPI(appid)
        self.weather_data = None

    def display_menu(self):
        print("\n===== Weather Forecast Application =====")
        print("1. Get current weather conditions")
        print("2. Get 5-day forecast")
        print("3. Exit")

    def get_user_input(self):
        user_input = input("Enter your choice (1-3): ")
        try:
            return int(user_input)
        except ValueError:
            return user_input

    def run(self):
        while True:
            self.display_menu()
            option = self.get_user_input()

            if option == 1:
                location_input = input("Enter city name or location (format: city or lat lon): ")
                city_id = self.weather_api.find_city(location_input)
                if city_id:
                    weather_info = self.weather_api.get_weather(city_id)
                    if weather_info:
                        self.weather_data = WeatherData(**weather_info)
                        self.weather_data.display_weather()
                    else:
                        print("No weather data found.")
                else:
                    print("Invalid city or location.")
            elif option == 2:
                location_input = input("Enter city name or location (format: city or lat lon): ")
                city_id = self.weather_api.find_city(location_input)
                if city_id:
                    weather5 = Weather5(appid)
                    weather5.get_weather_5(city_id)
                else:
                    print("Invalid city or location.")
            elif option == 3:
                print("Exiting the Weather app. Goodbye!")
                break
            else:
                print("Invalid option. Please enter a number between 1 and 3.")

weather_app = WeatherApp(appid)
weather_app.run()




===== Weather Forecast Application =====
1. Get current weather conditions
2. Get 5-day forecast
3. Exit


Enter your choice (1-3):  1
Enter city name or location (format: city or lat lon):  Kyiv


Temperature: -2.23°C
Description: overcast clouds
Humidity: 94%
Wind Speed: 1.04 km/h

===== Weather Forecast Application =====
1. Get current weather conditions
2. Get 5-day forecast
3. Exit


Enter your choice (1-3):  2
Enter city name or location (format: city or lat lon):  Kyiv


5-Day Forecast:
Day: 2023-12-09 12:00:00, Temperature: -2.74°C, Description: overcast clouds
Day: 2023-12-10 12:00:00, Temperature: 0.27°C, Description: overcast clouds
Day: 2023-12-11 12:00:00, Temperature: 0.43°C, Description: overcast clouds
Day: 2023-12-12 12:00:00, Temperature: 0.86°C, Description: overcast clouds
Day: 2023-12-13 12:00:00, Temperature: 1.93°C, Description: broken clouds

===== Weather Forecast Application =====
1. Get current weather conditions
2. Get 5-day forecast
3. Exit


Enter your choice (1-3):  3


Exiting the Weather app. Goodbye!


In [1]:
#Importing libraries
from tkinter import *
import requests
from tkinter import messagebox 
# Location, Temperature, Description, Humidity, Wind speed
# API key: b5c8d5d8a33d41eaacac223656d960f2
appid = 'b5c8d5d8a33d41eaacac223656d960f2'

# Classes:

# Super class WeatherAPI, which send request to the API, finds the city and gets the weather
class WeatherAPI:
    def __init__(self, appid):
        self.appid = appid #The only parameter it takes is the API ID

    def find_city(self, location_input):
        try:
            if ' ' in location_input: 
                #If the location has space and there are numbers, the program is looking for the coordinates
                lat, lon = map(int, location_input.split())
                
                #Initialise the variable for request with link for API and parameters
                res = requests.get("http://api.openweathermap.org/data/2.5/find",
                                   params={
                                       'lat': lat,#Latitude
                                       'lon': lon,#Longitude
                                       'type': 'like',#This means that we can enter the name in any language
                                       'units': 'metric',#Degrees celcius instead of Farenheit
                                       'APPID': self.appid #API ID
                                   })
                #If it is just a text, we are looking for the city
            else:
                #If the location is just a text, the programm is going to find city by its name
                res = requests.get("http://api.openweathermap.org/data/2.5/find",
                                   params={
                                       #The same parameters but with city, not lot and lat
                                       'q': location_input,
                                       'type': 'like',
                                       'units': 'metric',
                                       'APPID': self.appid
                                   })

            data = res.json()#We attach the json type requst into a variable
            
            #If everyting is correct, we are going to return the proper weather data
            return data['list'][0]['id'] if data['list'] else None
        
        #exception if we do not find the city or location
        except Exception as e:
            print("Exception (find):", e)
            return None

        #The function for printing the weather from the requests
        #When we found the city, we can now get the weather
    def get_weather(self, city_id):
        try:
            #The request is the same as for find_city
            res = requests.get("http://api.openweathermap.org/data/2.5/weather",
                               params={
                                   'id': city_id,
                                   'units': 'metric',
                                   'lang': 'en',
                                   'APPID': self.appid
                               })
            data = res.json()
            return {
                #Returning data
                'temperature': data['main']['temp'],
                'description': data['weather'][0]['description'],
                'humidity': data['main']['humidity'],
                'wind_speed': data['wind']['speed']
            }
        except Exception as e:
            print("Exception (weather):", e)
            return None

        
#Using single inheritance and polymorphism, we create the class for 5-day forecats, that takes the APP ID 
#and has the ovverriding methods
class Weather5(WeatherAPI):
    def __init__(self, appid):
        super().__init__(appid)

    def get_weather_5(self, city_id):
        try:
            res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                               params={
                                   'id': city_id,
                                   'units': 'metric',
                                   'lang': 'en',
                                   'APPID': self.appid
                               })
            
            data = res.json()
            print("5-Day Forecast:")
            unique_days = set()
            #Using the loop for printing the data in 5-day forecast
            for day in data['list']:
                # Check if the time is 12:00:00
                if day['dt_txt'].endswith('12:00:00'):
                    date = day['dt_txt'].split(' ')[0]
                    if date not in unique_days:
                        unique_days.add(date)
                        
                        print(f"Day: {day['dt_txt']}, Temperature: {day['main']['temp']}°C, Description: {day['weather'][0]['description']}")
        except Exception as e:
            print("Exception (forecast):", e)

#Cleaing labels for GUI
def clear_labels(labels_to_clear):
    # Loop through labels and destroy them
    for label in labels_to_clear:
        label.destroy()
#Checking if the city is in Russia
def is_city_in_russia(city_data):
    # Check if the country code is 'RU' (Russia)
    return city_data.get('sys', {}).get('country') == 'RU'

def show_GUI():
    #Taking the city ID from the Entrybox of GUI
    cityID = entry.get()

    try:
        #Getting the request
        res = requests.get("http://api.openweathermap.org/data/2.5/weather",
                           params={
                               'id': weather_app.find_city(cityID),#As for ID, we use find city method from weatherAPI class
                               'units': 'metric',
                               'lang': 'en',
                               'APPID': appid
                           })
        data = res.json()

        #checking if the city is in Russia
        if is_city_in_russia(data):
            messagebox.showinfo("Stand with Ukraine", "Forecast is not available for the agressor country")
            #sending the message alert
            return



        #updating labels for 5-day forecast
        label_day1.set("")
        label_day2.set("")
        label_day3.set("")
        label_day4.set("")
        label_day5.set("")

        #Updating labels for 1-day forecast
        label_temp.set(f"Temperature: {data['main']['temp']}°C")
        label_description.set(f"Description: {data['weather'][0]['description']}")
        label_humidity.set(f"Humidity: {data['main']['humidity']}%")
        label_wind_speed.set(f"Wind Speed: {data['wind']['speed']}km/h")

    except Exception as e:
        print("Exception (weather):", e)
        messagebox.showinfo("Unexpected error", "Enter the proper location")
        return None

def show_GUI_5():
    cityID = entry.get()
    try:
        #Getting the request
        city_data = requests.get("http://api.openweathermap.org/data/2.5/weather",
                                 params={
                                     'id': weather_app.find_city(cityID),
                                     'units': 'metric',
                                     'lang': 'en',
                                     'APPID': appid
                                 }).json()

        #Cchecking if the city is in Russia
        if is_city_in_russia(city_data):
            #sending the message alert
            messagebox.showinfo("Stand with Ukraine", "Forecast is not available for the agressor country")
            return

        # Fetch the 5-day forecast
        res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                           params={
                               'id': weather_app.find_city(cityID),#As for ID, we use find city method from weatherAPI class
                               'units': 'metric',
                               'lang': 'en',
                               'APPID': appid
                           })
        data = res.json()

        unique_days = set()
        for day in data['list']:
            # Check if the time is 12:00:00
            if day['dt_txt'].endswith('12:00:00'):
                date = day['dt_txt'].split(' ')[0]
                if date not in unique_days:
                    unique_days.add(date)
                    print(f"Day: {day['dt_txt']}, Temperature: {day['main']['temp']}°C, Description: {day['weather'][0]['description']}")

        #updating labels for 1-day forecast
        label_temp.set("")
        label_description.set("")
        label_humidity.set("")
        label_wind_speed.set("")

        #updating labels for 5-day forecast
        label_day1.set(f"Day1: {data['list'][0]['main']['temp']}°C {data['list'][0]['weather'][0]['description']}")
        label_day2.set(f"Day2: {data['list'][1]['main']['temp']}°C {data['list'][1]['weather'][0]['description']}")
        label_day3.set(f"Day3: {data['list'][2]['main']['temp']}°C {data['list'][2]['weather'][0]['description']}")
        label_day4.set(f"Day4: {data['list'][3]['main']['temp']}°C {data['list'][3]['weather'][0]['description']}")
        label_day5.set(f"Day5: {data['list'][4]['main']['temp']}°C {data['list'][4]['weather'][0]['description']}")
    #exception if the city was not found
    except Exception as e:
        print("Exception (forecast):", e)
        messagebox.showinfo("Unexpected error", "Enter the proper location")#sending the message alert

# Create the root window
root = Tk()
root['bg'] = "#ffffff"
root.title("Weather Forecast")
root.geometry('700x700')
root.resizable(width=False, height=False)

weather_app = Weather5(appid)#Creating the object of a Weather5 class that takes appid

#Welcome_message
welcome_message = Label(
    root,
    text="Welcome to Weather Forecast (Artem, Femi, Zifan)",
    foreground="black",
    background="white",
    font=("Arial", 25)
)
welcome_message.place(x=20, y=20)

#Enter city or location message :
city_enter = Label(
    root,
    text="Enter city or location:",
    foreground="black",
    background="white",
    font=("Arial", 25)
)
city_enter.place(x=20, y=150)

#User entry
entry = Entry(
    root,
    foreground="black",
    background="white",
    font=("Arial", 20))
entry.place(x=20, y=190)

#We need to initialise variables for 1-day and 5-day forecast, otherwise we cant clean them and set new

#Variables for 1-day forecast
label_temp = StringVar()
label_description = StringVar()
label_humidity = StringVar()
label_wind_speed = StringVar()

#Variables for 5-day forecast
label_day1 = StringVar()
label_day2 = StringVar()
label_day3 = StringVar()
label_day4 = StringVar()
label_day5 = StringVar()

#Labels for 1-day forecast
Label(root, textvariable=label_temp, foreground="black", background="white", font=("Arial", 25)).place(x=22, y=320)
Label(root, textvariable=label_description, foreground="black", background="white", font=("Arial", 25)).place(x=22, y=350)
Label(root, textvariable=label_humidity, foreground="black", background="white", font=("Arial", 25)).place(x=22, y=380)
Label(root, textvariable=label_wind_speed, foreground="black", background="white", font=("Arial", 25)).place(x=22, y=410)

#Labels for 5-day forecast
Label(root, textvariable=label_day1, foreground="black", background="white", font=("Arial", 25)).place(x=20, y=320)
Label(root, textvariable=label_day2, foreground="black", background="white", font=("Arial", 25)).place(x=20, y=350)
Label(root, textvariable=label_day3, foreground="black", background="white", font=("Arial", 25)).place(x=20, y=380)
Label(root, textvariable=label_day4, foreground="black", background="white", font=("Arial", 25)).place(x=20, y=410)
Label(root, textvariable=label_day5, foreground="black", background="white", font=("Arial", 25)).place(x=20, y=440)

#Buttons that call show_GUI and show_GUI_5 functions
show_weather1 = Button(root, text="1 day forecast", command=show_GUI)
show_weather1.place(x=20, y=250)

show_weather5 = Button(root, text="5 day forecast", command=show_GUI_5)
show_weather5.place(x=150, y=250)

root.mainloop()  # Tkinter method, without it, the code doesn't run

Exception (find): invalid literal for int() with base 10: 'Kyiv'
Exception (weather): 'main'
Day: 2023-12-09 12:00:00, Temperature: -2.74°C, Description: overcast clouds
Day: 2023-12-10 12:00:00, Temperature: 0.27°C, Description: overcast clouds
Day: 2023-12-11 12:00:00, Temperature: 0.43°C, Description: overcast clouds
Day: 2023-12-12 12:00:00, Temperature: 0.86°C, Description: overcast clouds
Day: 2023-12-13 12:00:00, Temperature: 1.93°C, Description: broken clouds
Day: 2023-12-09 12:00:00, Temperature: -2.74°C, Description: overcast clouds
Day: 2023-12-10 12:00:00, Temperature: 0.27°C, Description: overcast clouds
Day: 2023-12-11 12:00:00, Temperature: 0.43°C, Description: overcast clouds
Day: 2023-12-12 12:00:00, Temperature: 0.86°C, Description: overcast clouds
Day: 2023-12-13 12:00:00, Temperature: 1.93°C, Description: broken clouds
Day: 2023-12-09 12:00:00, Temperature: -2.21°C, Description: light snow
Day: 2023-12-10 12:00:00, Temperature: -2.48°C, Description: overcast clouds
